<a href="https://colab.research.google.com/github/akhilasree26/Online-Resume/blob/main/summerprjt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
usgs_earthquake_database_path = kagglehub.dataset_download('usgs/earthquake-database')

print('Data source import complete.')


100%|██████████| 590k/590k [00:00<00:00, 18.5MB/s]

Extracting files...
Data source import complete.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf

In [3]:
data=pd.read_csv('/kaggle/input/earthquake-database/database.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/earthquake-database/database.csv'

In [ ]:
data

In [ ]:
data.info()

#Data Preprocessing

In [ ]:
# looking at missing values
data.isna()

In [ ]:
#no. of missing values in each coloumn
data.isna().sum()

In [ ]:
#taking it as mean to see in %
data.isna().mean()

In [ ]:
data.isna().sum()>0.66*data.shape[0]

In [ ]:
data.loc[:,data.isna().sum()>0.66*data.shape[0]]

In [ ]:
data.loc[:,data.isna().sum()>0.66*data.shape[0]].columns

In [ ]:
null_columns=data.loc[:,data.isna().sum()>0.66*data.shape[0]].columns

In [ ]:
data= data.drop(null_columns,axis=1)

In [ ]:
data.isna().sum()

In [ ]:
data['Root Mean Square']=data['Root Mean Square'].fillna(data['Root Mean Square'].mean())

In [ ]:
data=data.dropna(axis=0).reset_index(drop=True)

In [ ]:
data.isna().sum().sum()

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# -------------------------
# Reproducibility
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

# -------------------------
# Load dataset (try kaggles path; else assume 'data' exists in kernel)
csv_path = '/kaggle/input/earthquake-database/database.csv'
if os.path.exists(csv_path):
    data = pd.read_csv(csv_path)
    print("Loaded CSV from", csv_path)
else:
    if 'data' in globals():
        print("Using existing 'data' variable from kernel.")
    else:
        raise FileNotFoundError(f"Could not find dataset at {csv_path} and variable 'data' not in globals().")

# -------------------------
# Minimal feature selection and cleaning
if 'Datetime' not in data.columns:
    if 'Date' in data.columns and 'Time' in data.columns:
        data['Datetime'] = pd.to_datetime(data['Date'].astype(str) + ' ' + data['Time'].astype(str), errors='coerce')
    elif 'time' in data.columns:
        data['Datetime'] = pd.to_datetime(data['time'], errors='coerce')
    else:
        data['Datetime'] = pd.to_datetime(pd.Series(range(len(data))), errors='coerce')

keep_cols = [c for c in ['Latitude','Longitude','Depth','Magnitude','Datetime'] if c in data.columns]
df = data[keep_cols].dropna().copy()
df = df.sort_values('Datetime').reset_index(drop=True)

print("Final dataframe shape (after selecting essential columns):", df.shape)
print("Columns used:", df.columns.tolist())

# -------------------------
# Scaling
feature_cols = ['Latitude','Longitude','Depth']
target_col = 'Magnitude'

feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

features = df[feature_cols].values.astype(np.float32)
target = df[[target_col]].values.astype(np.float32)

features_scaled = feature_scaler.fit_transform(features)
target_scaled = target_scaler.fit_transform(target)

# -------------------------
# Create supervised sequences
def create_feature_sequences(X_feat, y_target, seq_len=20):
    Xs, ys = [], []
    for i in range(len(X_feat) - seq_len):
        Xs.append(X_feat[i:i+seq_len])
        ys.append(y_target[i+seq_len])
    return np.array(Xs), np.array(ys).reshape(-1,1)

SEQ_LEN = 20
X, y = create_feature_sequences(features_scaled, target_scaled, seq_len=SEQ_LEN)
print("Sequence data shape:", X.shape, y.shape)

# -------------------------
# Train-test split
test_size = 0.2
split_idx = int((1 - test_size) * len(X))
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

print("Train samples:", X_train.shape[0], "Test samples:", X_test.shape[0])

# -------------------------
# CNN-LSTM model
n_features = X_train.shape[2]
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(SEQ_LEN, n_features)),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    LSTM(100, return_sequences=False),
    Dropout(0.3),
    Dense(50, activation='relu'),
    Dense(1, activation='linear')
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

es = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=60,
    batch_size=32,
    callbacks=[es],
    verbose=2
)

# -------------------------
# Evaluate CNN-LSTM
y_pred_scaled = model.predict(X_test)
y_pred = target_scaler.inverse_transform(y_pred_scaled)
y_test_orig = target_scaler.inverse_transform(y_test)

# Calculate metrics
mse = mean_squared_error(y_test_orig, y_pred)
mae = mean_absolute_error(y_test_orig, y_pred)
rmse = np.sqrt(mse)

print(f"\n📊 Hybrid CNN-LSTM Metrics")
print(f"MSE  : {mse:.6f}")
print(f"MAE  : {mae:.6f}")
print(f"RMSE : {rmse:.6f}")

# Plot Actual vs Predicted
plt.figure(figsize=(12,5))
plt.plot(y_test_orig.flatten(), label='Actual Magnitude', color='orange', linewidth=1)
plt.plot(y_pred.flatten(), label='Predicted Magnitude', color='blue', linewidth=1)
plt.title('Hybrid CNN-LSTM: Actual vs Predicted (Test set)')
plt.xlabel('Test sample index')
plt.ylabel('Magnitude')
plt.legend()
plt.show()

# Plot training history
plt.figure(figsize=(10,4))
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.legend()
plt.show()

# -------------------------
# Univariate LSTM
def create_univariate_sequences(y_vec, seq_len=20):
    Xu, yu = [], []
    for i in range(len(y_vec) - seq_len):
        Xu.append(y_vec[i:i+seq_len, 0])
        yu.append(y_vec[i+seq_len, 0])
    return np.array(Xu), np.array(yu)

X_u, y_u = create_univariate_sequences(target_scaled, seq_len=SEQ_LEN)
X_u = X_u.reshape((X_u.shape[0], X_u.shape[1], 1))
y_u = y_u.reshape(-1,1)

split_idx_u = int((1 - test_size) * len(X_u))
X_u_train, X_u_test = X_u[:split_idx_u], X_u[split_idx_u:]
y_u_train, y_u_test = y_u[:split_idx_u], y_u[split_idx_u:]

uni_model = Sequential([
    LSTM(64, input_shape=(SEQ_LEN,1)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])
uni_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
uni_model.summary()

es2 = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
uni_history = uni_model.fit(X_u_train, y_u_train, validation_data=(X_u_test,y_u_test),
                           epochs=40, batch_size=32, callbacks=[es2], verbose=2)

# Evaluate univariate LSTM
y_u_pred_scaled = uni_model.predict(X_u_test)
y_u_pred = target_scaler.inverse_transform(y_u_pred_scaled)
y_u_test_orig = target_scaler.inverse_transform(y_u_test)

mse_u = mean_squared_error(y_u_test_orig, y_u_pred)
mae_u = mean_absolute_error(y_u_test_orig, y_u_pred)
rmse_u = np.sqrt(mse_u)

print(f"\n📊 Univariate LSTM Metrics")
print(f"MSE  : {mse_u:.6f}")
print(f"MAE  : {mae_u:.6f}")
print(f"RMSE : {rmse_u:.6f}")

# Forecast future
def forecast_future_univariate(model, recent_scaled_mags, n_steps=10):
    seq = recent_scaled_mags.copy().tolist()
    preds = []
    for _ in range(n_steps):
        x = np.array(seq[-SEQ_LEN:]).reshape(1, SEQ_LEN, 1)
        p = model.predict(x)[0,0]
        preds.append(p)
        seq.append(p)
    return np.array(preds)

recent_seq = target_scaled[-SEQ_LEN:, 0]
N_FUTURE = 20
future_preds_scaled = forecast_future_univariate(uni_model, recent_seq, n_steps=N_FUTURE)
future_preds = target_scaler.inverse_transform(future_preds_scaled.reshape(-1,1)).flatten()

print(f"\nNext {N_FUTURE} forecasted magnitudes (univariate LSTM):")
print(np.round(future_preds, 3))

hist_mag = df[target_col].values[-200:]
plt.figure(figsize=(12,5))
plt.plot(range(len(hist_mag)), hist_mag, label='Recent historical magnitudes')
plt.plot(range(len(hist_mag)-1, len(hist_mag)-1+N_FUTURE+1), np.concatenate([[hist_mag[-1]], future_preds]),
         label='Forecasted magnitudes', marker='o')
plt.title('Historical (last 200) and Forecasted Magnitudes')
plt.xlabel('Index (time-ordered events)')
plt.ylabel('Magnitude')
plt.legend()
plt.show()

print("✅ Done.")


In [ ]:
#best algorithim is CNN-LSTM (as it got lowesr rmse score)
# Model             MSE       MAE     RMSE
Hybrid CNN-LSTM   0.0231    0.119   0.152
Univariate LSTM   0.0315    0.143   0.177


In [ ]:
# CNN-LSTM seismic forecasting with different epoch settings

import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# -------------------------
# Reproducibility
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

# -------------------------
# Load dataset
csv_path = '/kaggle/input/earthquake-database/database.csv'
data = pd.read_csv(csv_path)

# -------------------------
# Create datetime if missing
if 'Datetime' not in data.columns:
    if 'Date' in data.columns and 'Time' in data.columns:
        data['Datetime'] = pd.to_datetime(data['Date'].astype(str) + ' ' + data['Time'].astype(str), errors='coerce')
    elif 'time' in data.columns:
        data['Datetime'] = pd.to_datetime(data['time'], errors='coerce')
    else:
        data['Datetime'] = pd.to_datetime(pd.Series(range(len(data))), errors='coerce')

# Select useful columns
df = data[['Latitude','Longitude','Depth','Magnitude','Datetime']].dropna().sort_values('Datetime').reset_index(drop=True)

# -------------------------
# Scale data
feature_cols = ['Latitude','Longitude','Depth']
target_col = 'Magnitude'
feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

features_scaled = feature_scaler.fit_transform(df[feature_cols].values.astype(np.float32))
target_scaled = target_scaler.fit_transform(df[[target_col]].values.astype(np.float32))

# -------------------------
# Create sequences
def create_sequences(X, y, seq_len=20):
    Xs, ys = [], []
    for i in range(len(X) - seq_len):
        Xs.append(X[i:i+seq_len])
        ys.append(y[i+seq_len])
    return np.array(Xs), np.array(ys).reshape(-1,1)

SEQ_LEN = 20
X, y = create_sequences(features_scaled, target_scaled, SEQ_LEN)

# Train-test split
test_size = 0.2
split_idx = int((1 - test_size) * len(X))
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

# -------------------------
# Function to build CNN-LSTM
def build_cnn_lstm(seq_len, n_features):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(seq_len, n_features)),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=32, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        LSTM(100),
        Dropout(0.3),
        Dense(50, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# -------------------------
# Train and evaluate for different epoch settings
results = {}
n_features = X_train.shape[2]

for epochs in [40, 60, 70, 80]:
    print(f"\nTraining CNN-LSTM with {epochs} epochs...")
    model = build_cnn_lstm(SEQ_LEN, n_features)
    es = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=0)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=epochs,
        batch_size=32,
        callbacks=[es],
        verbose=0
    )

    # Predictions
    y_pred_scaled = model.predict(X_test, verbose=0)
    y_pred = target_scaler.inverse_transform(y_pred_scaled)
    y_test_orig = target_scaler.inverse_transform(y_test)

    # Metrics
    mse = mean_squared_error(y_test_orig, y_pred)
    mae = mean_absolute_error(y_test_orig, y_pred)
    rmse = np.sqrt(mse)
    results[epochs] = {"MSE": mse, "MAE": mae, "RMSE": rmse}

    # Plot
    plt.figure(figsize=(10,4))
    plt.plot(y_test_orig.flatten(), label='Actual')
    plt.plot(y_pred.flatten(), label=f'Predicted ({epochs} epochs)')
    plt.title(f'CNN-LSTM (Epochs={epochs}) - Actual vs Predicted')
    plt.legend()
    plt.show()

# -------------------------
# Show results table
print("\nPerformance Comparison (CNN-LSTM):")
df_results = pd.DataFrame(results).T
print(df_results)
